In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline

In [ ]:
data_path = '../data/'
listing_info_fn = 'listing_info.csv'
test_fn = 'test.csv'
train_fn = 'train.csv'
user_behavior_logs_fn = 'user_behavior_logs.csv'
user_info_fn = 'user_info.csv'
user_repay_logs_fn  = 'user_repay_logs.csv'
user_taglist_fn = 'user_taglist.csv'

preprocess_listing_info_fn = 'preprocess_listing_info.csv'
preprocess_test_fn = 'preprocess_test.csv'
preprocess_train_fn = 'preprocess_train.csv'
preprocess_user_behavior_logs_fn = 'preprocess_user_behavior_logs.csv'
preprocess_user_info_fn = 'preprocess_user_info.csv'
preprocess_user_repay_logs_fn  = 'preprocess_user_repay_logs.csv'
preprocess_user_taglist_fn = 'preprocess_user_taglist.csv'

preprocess_train_data_listing_info_fn = 'preprocess_train_data_listing_info.csv'
preprocess_user_info_new_fn = 'preprocess_user_info_new.csv'

In [ ]:
user_info = pd.read_csv(data_path+preprocess_user_info_fn)
user_info.drop(['Unnamed: 0'], axis=1,inplace=True)
user_info_copy = user_info.copy(deep=True)
user_info_copy.head()

In [ ]:
handled_index_list = []
join_serise_list = []
count = 0
join_index = ['index', 'insertdata_first', 'insertdata_last', 'insertdata_times']
join_serise = pd.Series(index=join_index)
for idx, row in user_info.iterrows():
    count += 1
    if count%1000 == 0:
        print(count)
    temp_user = user_info_copy.loc[user_info_copy['user_id'] == row['user_id']]
    temp_user_insterdata_times = len(temp_user)
    if temp_user_insterdata_times == 0:
        print('error')
        break
    if idx in handled_index_list:
        continue
    if temp_user_insterdata_times == 1:   
        temp_series = pd.Series([idx, temp_user['insert_ts'].values[0], temp_user['insert_ts'].values[0], 1], index=join_index)
        join_serise_list.append(temp_series)
    else:
        handled_index_list.append(idx)
        temp_user_ts = temp_user['insert_ts']
        temp_user_ts_max = temp_user_ts.max()
        temp_user_ts_min = temp_user_ts.min()
        temp_user_ts_dict = {v:k for k, v in temp_user_ts.to_dict().items()}
        temp_user_ts_max_idx = temp_user_ts_dict[temp_user_ts_max]
        temp_user_index_list = temp_user.index.to_list()
        temp_user_index_list.remove(temp_user_ts_max_idx)
        user_info_copy.drop(index=temp_user_index_list, axis=0, inplace=True)
        temp_series = pd.Series([temp_user_ts_max_idx, temp_user_ts_min, temp_user_ts_max, temp_user_insterdata_times], index=join_index)
        join_serise_list.append(temp_series)
        handled_index_list.extend(temp_user.index.values.tolist())
#         break

join_serise = pd.concat(join_serise_list, axis=1)
join_serise.head()

In [ ]:
join_serise = join_serise.T
print(join_serise.shape)
join_serise.head()

In [ ]:
join_serise.to_csv(data_path+'preprocess_join_serise.csv', index=False, encoding='utf-8')

In [ ]:
join_serise.set_index(['index'], inplace=True)
join_serise.head()

In [ ]:
join_serise.drop(index=0, inplace=True)
join_serise['index'] = join_serise['index'].astype(dtype='int64',copy=False)
join_serise.set_index(['index'], inplace=True)

In [ ]:
user_info_copy_new = pd.merge(user_info_copy, join_serise, left_index=True, right_index=True)

In [ ]:
user_info_copy_new.drop(['insertdate', 'insert_ts'], axis=1,inplace=True)
user_info_copy_new.head()

In [ ]:
user_info_copy_new.to_csv(data_path+preprocess_user_info_new_fn, index=False, encoding='utf-8')